In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import glob
import os
import numpy as np
import json
import cv2

dataDir = '/home/ubuntu/Data/autoFrameFilter/test'

def draw_one_alphapose(frame, oneResult, format='coco'):
    if format == 'coco':
        l_pair = [(0, 1), (0, 2), (1, 3), (2, 4),  # Head
            (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),(17, 11), (17, 12),  # Body
            (11, 13), (12, 14), (13, 15), (14, 16)]
        p_color = [(0, 255, 255),(0, 191, 255),(0, 255, 102),(0, 77, 255), (0, 255, 0), 
                   #Nose, LEye, REye, LEar, REar
                  (77,255,255),(77, 255, 204), (77,204,255), (191, 255, 77), (77,191,255), (191, 255, 77), 
                   #LShoulder, RShoulder, LElbow, RElbow, LWrist, RWrist
                  (204,77,255),(77,255,204), (191,77,255), (77,255,191), (127,77,255), (77,255,127), (0, 255, 255)] 
                   #LHip, RHip, LKnee, Rknee, LAnkle, RAnkle, Neck
        line_color = [(0, 215, 255), (0, 255, 204), (0, 134, 255), (0, 255, 50), 
                    (77,255,222), (77,196,255), (77,135,255), (191,255,77), (77,255,77), 
                    (77,222,255), (255,156,127), 
                    (0,127,255), (255,127,77), (0,77,255), (255,77,36)]
    img = frame
    part_line = {}
    kp_preds = np.array(oneResult['keypoints'])
    kp_scores = np.array(oneResult['kp_score'])
    # Draw keypoints
    for n in range(len(kp_scores)):
        if kp_scores[n] <= 0.05:
            continue
        cor_x, cor_y = int(kp_preds[n, 0]), int(kp_preds[n, 1])
        part_line[n] = (cor_x, cor_y)
        cv2.circle(img, (cor_x, cor_y), 4, p_color[n], -1)
    # Draw limbs
    for i, (start_p, end_p) in enumerate(l_pair):
        if start_p in part_line and end_p in part_line:
            start_xy = part_line[start_p]
            end_xy = part_line[end_p]
            cv2.line(img, start_xy, end_xy, line_color[i], 2*(kp_scores[start_p] + kp_scores[end_p]) + 1)
    return img

def cut_one_alphapose(frame, oneResult):
    img=frame
    kp_preds = np.array(oneResult['keypoints'])
    kp_scores = np.array(oneResult['kp_score'])
    h,w,_=img.shape
    #print(img.shape)
    minX, minY =w,h
    maxX, maxY =0,0
    for n in range(len(kp_scores)):
        if kp_scores[n] <= 0.05:
            continue
        minX = min(kp_preds[n,0], minX)
        minY = min(kp_preds[n,1], minY)
        maxX = max(kp_preds[n,0], maxX)
        maxY = max(kp_preds[n,1], maxY)
    assert maxX>minX, print('error : maxX<=minX')
    assert maxY>minY, print('error : maxY<=minY')
    cutW=maxX-minX
    cutH=maxY-minY
    minX = int(max(0,minX-0.3*cutW))
    maxX = int(min(w,maxX+0.3*cutW))
    minY = int(max(0,minY-0.3*cutH))
    maxY = int(min(h,maxY+0.3*cutH))
    imgCut = img[minY:maxY, minX:maxX]
    cutH, cutW, _ = imgCut.shape
    maxCutW, maxCutH = 711, 300
    if cutH/cutW >maxCutH/maxCutW:
        reH = maxCutH
        reW = int(maxCutH*cutW/cutH)
    else:
        reW = maxCutW
        reH = int(maxCutW*cutH/cutW)
    imgCut = cv2.resize(imgCut,(reW,reH))
    
    return imgCut





def callback(cateNum, cateName):
    try:
        imgNameLabel.configure(text='%s     img: %d/%d     human: %d/%d'
                               %(imgPathVar.get(), imgCountVar.get(), imgNumVar.get(), humanCountVar.get(), humanNumVar.get()))
        with open(annPathVar.get(),'r',encoding='utf-8') as jsonfile:
            annResult = json.load(jsonfile)
        annResult[humanCountVar.get()]['categorie']=cateNum
        annResult[humanCountVar.get()]['categorieName']=cateName
        with open(annPathVar.get(),'w',encoding='utf-8') as jsonfile:
            json.dump(annResult,jsonfile)
        photo, photoCut = next(show)
        imgLabel.configure(image=photo)
        imgCutLabel.configure(image=photoCut)
        
    except:
        print('end')
    
        
def show_img(dataDir):
    if not os.path.exists('./temp'):
        os.makedirs('./temp')
    imgDir = os.path.join(dataDir,'images')
    kpDir = os.path.join(dataDir, 'keypoints')
    annDir = os.path.join(dataDir, 'annotations')
    if not os.path.exists(annDir):
        os.makedirs(annDir)
    imgPathList = glob.glob(os.path.join(imgDir,'*.jpg'))
    for imgCount, imgPath in enumerate(imgPathList):
        dirPath, imgName = os.path.split(imgPath)
        kpName = imgName[:-4]+'_keypoints.json'
        kpPath = os.path.join(kpDir,kpName)
        annName= imgName[:-4]+'.json'
        annPath = os.path.join(annDir,annName)
        
        imgPathVar.set(imgPath)
        annPathVar.set(annPath)
        imgCountVar.set(imgCount)
        imgNumVar.set(len(imgPathList))
        with open(kpPath,'r',encoding='utf-8') as jsonfile:
            result = json.load(jsonfile)
        with open(annPath,'w',encoding='utf-8') as jsonfile:
            json.dump(result,jsonfile)
        for humanCount, humanKp in enumerate(result):
            img = cv2.imread(imgPath)
            imgCut = cut_one_alphapose(img, humanKp)
            cv2.imwrite('./temp/temp.jpg', imgCut)
            imgCut = Image.open('./temp/temp.jpg')  
            imgCut = ImageTk.PhotoImage(imgCut)
            
            imgKp = draw_one_alphapose(img, humanKp)
            cv2.imwrite('./temp/temp.jpg', imgKp)
            imgKp = Image.open('./temp/temp.jpg')  
            imgKp = imgKp.resize((711,400))
            imgKp = ImageTk.PhotoImage(imgKp)
           
            humanCountVar.set(humanCount)
            humanNumVar.set(len(result))
            yield imgKp, imgCut
        print('%s is completed'%imgName)
            
root = tk.Tk()
root.title('test')

#define vars
imgPathVar = tk.StringVar()
imgCountVar = tk.IntVar()
imgNumVar = tk.IntVar()
humanNumVar = tk.IntVar()
humanCountVar = tk.IntVar()
annPathVar = tk.StringVar()

#show img frame
frameImg = tk.Frame(root)
show_img(dataDir)
show = show_img(dataDir)
photo, photoCut = next(show)
imgLabel = tk.Label(frameImg, image=photo, width=711, height=400)
imgLabel.pack()
imgCutLabel = tk.Label(frameImg, image=photoCut, width=711, height=300)
imgCutLabel.pack(side=tk.BOTTOM)

#label frame
frameInfo = tk.Frame(root)
imgNameLabel  = tk.Label(frameInfo, text='%s     img: %d/%d     human: %d/%d'
                         %(imgPathVar.get(), imgCountVar.get(), imgNumVar.get(), humanCountVar.get(), humanNumVar.get()))
imgNameLabel.pack()

#button frame
frameButton = tk.Frame(root)
theButton1 = tk.Button(frameButton, text="0: stand up   ", height=4, activebackground = 'GREEN', command=lambda:callback(0,'stand up'))
theButton1.pack()
theButton2 = tk.Button(frameButton, text="1: read&write ", height=4, activebackground = 'GREEN', command=lambda:callback(1,'read&write'))
theButton2.pack()
theButton3 = tk.Button(frameButton, text="2: listening  ", height=4, activebackground = 'GREEN', command=lambda:callback(2,'listening'))
theButton3.pack()
theButton4 = tk.Button(frameButton, text="3: hand up    ", height=4, activebackground = 'GREEN', command=lambda:callback(3,'hand up'))
theButton4.pack()
theButton5 = tk.Button(frameButton, text="4: turn around", height=4, activebackground = 'GREEN', command=lambda:callback(4,'turn around'))
theButton5.pack()
theButton6 = tk.Button(frameButton, text="5: sleep      ", height=4, activebackground = 'GREEN', command=lambda:callback(5,'sleep'))
theButton6.pack()
theButton7 = tk.Button(frameButton, text="6: else       ", height=4, activebackground = 'GREEN', command=lambda:callback(6,'else'))
theButton7.pack()

frameInfo.pack(side=tk.TOP)
frameImg.pack(side=tk.LEFT)
frameButton.pack(side=tk.RIGHT)


#start
root.mainloop()

test_29000_50.jpg is completed
test_34625_49.jpg is completed
